### Imports

In [2]:
import os
import pickle
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.utils import Sequence

import Levenshtein as Lev
import sys

In [3]:
print(tf.__version__)

2.0.0


To Decide
1. Batch Norm vs Layer Norm or No Norm for attention model
2. Autoencoder (CNN vs Vanilla vs None)
3. Shuffle or No

### Paths

In [4]:
train_path = "./LibriSpeech100/train/train_all/"
dev_path = "./LibriSpeech100/dev/dev_all/"
test_path = "./LibriSpeech100/test/test_all/"

## AutoEncoder

### Custom Data Generator for AutoEncoder

In [12]:
class DataGenerator_AutoEncoder(Sequence):
    def __init__(self, path, to_fit = True):
        self.path = path
        self.list_X = self.getList()
        self.to_fit = to_fit
    
    def __len__(self):
        return len(self.list_X)
    
    def __getitem__(self, index):      
        dict_X = self.get_dict_X(index)   
        X = np.array(self.generate_X(dict_X))
        
        if self.to_fit:    
            return X, X
        
        return X
    
    def getList(self):
        train_list = os.listdir(self.path)
        list_X = [item for item in train_list if item.split(".")[-1] == 'pkl']
        return list_X
    
    def get_dict_X(self, index):
        file_name = self.path + self.list_X[index]
        with open(file_name, 'rb') as pickle_file:
            dict_X = pickle.load(pickle_file)
        return dict_X
    
    def generate_X(self, dict_X):
        X = [value for key, value in dict_X.items()]
        return X

### Model Definition

In [13]:
def autoencoder():
    model = tf.keras.Sequential()
    model.add(layers.Dense(350))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(200))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(100))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(200))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(350))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(513))    
    return model

### Create, Compile and Run Model

In [14]:
model = autoencoder()
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.MeanSquaredError())
training_generator = DataGenerator_AutoEncoder(train_path)
validation_generator = DataGenerator_AutoEncoder(dev_path)
model.fit_generator(generator=training_generator, validation_data=validation_generator,epochs=20)

Epoch 1/20


ValueError: could not broadcast input array from shape (20,583) into shape (20)

## Main DNN

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, path, to_fit = True):
        self.path = path
        self.list_X, self.list_Y = self.getLists()
        self.to_fit = to_fit
    
    def __len__(self):
        return len(self.list_X)
    
    def __getitem__(self, index):      
        dict_X = self.get_dict_X(index)   
        
        if self.to_fit:
            dict_Y = self.get_dict_Y(index)
            X, Y = self.generate_XY(dict_X, dict_Y)
            return X, Y
        
        X = self.generate_X(dict_X)
        return X
    
    def getLists(self):
        list_X = []
        list_Y = []
        for item in sorted(os.listdir(self.path)):
            ext = item.split(".")[-1]
            if ext == 'pkl':
                list_X.append(item)
            elif ext == 'txt':
                list_Y.append(item)
        return list_X, list_Y
    
    def get_dict_X(self, index):
        file_name = self.path + self.list_X[index]
        with open(file_name, 'rb') as pickle_file:
            dict_X = pickle.load(pickle_file)
        return dict_X
    
    def get_dict_Y(self, index):
        file = open(train_path+filename)
        dict_Y = {}
        for line in file:
            data = line.split()
            key = data[0]
            value = ' '.join(data[1:])
            dict_Y[key] = value
        return dict_Y
    
    def generate_XY(self, dict_X, dict_Y):
        X = []
        Y = []
        for key in dict_X:
            X.append(dict_X[key])
            Y.append(dict_Y[key])
        return X, Y
    
    def generate_X(self, dict_X):
        X = [value for key, value in dict_X.items()]
        return X

In [ ]:
def wer(s1, s2):

    s1 =s1.lower()
    s2 =s2.lower()
    b = set(s1.lower().split() + s2.lower().split())
    
    word2char = dict(zip(b, range(len(b))))


    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    return Lev.distance(''.join(w1), ''.join(w2))/float(len(s2.split()))